In [ ]:
from sepal_ui import widgetFactory as wf
from sepal_ui import widgetBinding as wb

#use a class to define your input and output in order to have mutable variables
class gfc_process_IO:
    def __init__(self):
        #set up your inputs
        self.threshold = None
        
        #set up output
        self.clip_map = None
        self.csv_file = None
        
gfc_process_io = gfc_process_IO()

In [ ]:
GFC_MESSAGE = 'click on "Run process" to launch the process on your GEE account'
NO_THRESHOLD = 'No threshold have been selected'
NO_AOI = 'No aoi have been provided, select one in step 1'

In [ ]:
#create an empty result tile that will be filled with displayable plot, map, links, text
id_ = "gfc_result_widget"
title = "Results"

gfc_results = wf.Tile(id_, title, inputs=['No results to display yet'])

In [ ]:
import ipyvuetify as v

#create the output alert 
gfc_process_output = wf.OutputWidget(GFC_MESSAGE)

#create the button that start your process
gfc_process_btn = wf.ProcessBtn('Run GFC analysis')

#create the widgets following ipyvuetify requirements 
gfc_process_slider = v.Slider(
    label= 'Threshold', 
    class_="mt-5", 
    thumb_label='always', 
    v_model=0,
    step=5
)

gfc_process_inputs = [gfc_process_slider]

#bind the widget to the inputs
wb.bind(gfc_process_slider, gfc_process_io, 'threshold', gfc_process_output)

#create a process tile
id_ = "gfc_process_widget"
title = 'gfc_Process'

gfc_process = wf.Tile(
    id_, 
    title, 
    btn=gfc_process_btn, 
    inputs=gfc_process_inputs, 
    output=gfc_process_output
)

In [ ]:
#bind the button to the process by writing a custom function
from scripts import run
from sepal_ui.scripts import utils
from functools import partial

def process_start(widget, event, data, output):
    
    #in order to write the results
    global gfc_results
    
    #debug
    #asset = 'users/bornToBeAlive/aoi_PU'
    #threshold = 20
    
    asset = gfc_aoi_IO.assetId
    threshold = gfc_process_io.threshold
    
    #toggle the loading button
    utils.toggleLoading(widget)
    
    #check asset 
    if not asset: 
        utils.displayIO(output, NO_AOI, alert_type='error')
        utils.toggleLoading(widget)
        return 
    
    if not gfc_process_io.threshold:
        utils.displayIO(output, NO_THRESHOLD, alert_type='error')
        utils.toggleLoading(widget)
        return 
    
    #launch any process you want
    clip_map, csv_file = run.gfc_analysis(asset, gfc_process_io.threshold, output)
    setattr(gfc_process_io, 'clip_map', clip_map)
    setattr(gfc_process_io, 'csv_file', csv_file)
    
    #display the results 
    result_layout = run.displayGfcResults(clip_map, csv_file, asset)
    gfc_results.children[0].children = [gfc_results.children[0].children[0]] + result_layout
    
    #toggle the loading button
    utils.toggleLoading(widget)

gfc_process_btn.on_event('click', partial(process_start, output=gfc_process_output))

In [ ]:
#this tiles will only be displayed if you launch voila from this file 
gfc_process

In [ ]:
#this tiles will only be displayed if you launch voila from this file 
gfc_results